for ibm cloud production environment

In [10]:
import dotenv
import os

from langchain.chains import LLMChain
from langchain import PromptTemplate

from genai.credentials import Credentials
from genai.extensions.langchain import LangChainInterface
from genai.schemas import GenerateParams
from genai.model import Model

# from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
# from ibm_watson_machine_learning.foundation_models import Model
# from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
# from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

In [11]:
dotenv.load_dotenv()

api_key = os.getenv("GENAI_KEY", None)
api_endpoint = os.getenv("GENAI_API", None)

creds = Credentials(api_key,api_endpoint)

params = GenerateParams(
    decoding_method="greedy",
    max_new_tokens=1000,
    min_new_tokens=15,
    stop_sequences=['\n\n']
).dict()

# api_key = os.getenv("API_KEY", None)
# project_id = os.getenv("PROJECT_ID", None)

# creds = {
#     "url"    : "https://us-south.ml.cloud.ibm.com",
#     "apikey" : api_key
# }

# params = {
#     GenParams.DECODING_METHOD:"greedy",
#     GenParams.MAX_NEW_TOKENS:1000,
#     GenParams.MIN_NEW_TOKENS:15
# }

In [12]:
# model = Model(ModelTypes.LLAMA_2_70B_CHAT,creds,params,project_id)
# model = Model(model="codellama/codellama-34b-instruct",credentials=creds, params=params)
model = Model(model="bigcode/starcoder",credentials=creds, params=params)
    

In [13]:
file = open("sample.sql", "r")
sql = file.readlines()

file = open("sample_translated.sql", "r")
sql_translated = file.readlines()

In [14]:
from sql_metadata import Parser

parser = Parser(''.join(sql_translated))
for qry in parser.subqueries.values():
    print(f"subquery- {qry}")

subquery- SELECT e.party_intl_nbr, q.dte_of_birth_incorporation, q.education_lvl, q.total_asset, q.mthly_income, q.blind_ind, q.impaired_ind, q.quest_id, q.quest_type, q.raq_chnl, q.sign_dte, q.sign_time, q.version, q.br_code, q.risk_status, CASE WHEN q.sign_dte <= date_add(DATE '?Month_End_Dte', interval -12 MONTH) THEN 'E' WHEN ROUND(date_diff(DATE '?Month_End_Dte', q.dte_of_birth_incorporation, DAY) / NUMERIC '365.25', 2, 'ROUND_HALF_EVEN') BETWEEN 18 AND NUMERIC '24.99' AND upper(q.quest_type) = 'RI' AND q.version >= 6 AND substr(q.quest_ans, 1, 1) = '1' THEN real_risk_lvl WHEN ROUND(date_diff(DATE '?Month_End_Dte', q.dte_of_birth_incorporation, DAY) / NUMERIC '365.25', 2, 'ROUND_HALF_EVEN') BETWEEN 25 AND NUMERIC '34.99' AND upper(q.quest_type) = 'RI' AND q.version >= 6 AND substr(q.quest_ans, 1, 1) = '2' THEN real_risk_lvl WHEN ROUND(date_diff(DATE '?Month_End_Dte', q.dte_of_birth_incorporation, DAY) / NUMERIC '365.25', 2, 'ROUND_HALF_EVEN') BETWEEN 35 AND NUMERIC '50.99' AND upp

In [ ]:
few_shot_prompt = f"""Input:
sel dtab1
Output:
SELECT dtab1
Input:                                                
when PI_Warning = 'Y' then 'Y'
Output:
WHEN upper(dtabw.pi_warning) = 'Y' THEN 'Y'
Input:
when PI_Warning is null then null else 'N' end  "Professional_Investor"
Output:
WHEN dtabw.pi_warning IS NULL THEN CAST(NULL as STRING) ELSE 'N' END AS professional_investor
Input:
when q.sign_dte <= add_months(date '?Month_End_Dte', -12) then 'E'                                         
Output:
WHEN q.sign_dte <= date_add(DATE '?Month_End_Dte', interval -12 MONTH) THEN 'E'
Input:
when (date '?Month_End_Dte'-dte_of_birth_incorporation)/365.25 between 18 and 24.99 and quest_type = 'RI' and version >= 6 and substr(q.quest_ans, 1,1) = '1' then real_risk_lvl
Output:
WHEN ROUND(date_diff(DATE '?Month_End_Dte', q.dte_of_birth_incorporation, DAY) / NUMERIC '365.25', 2, 'ROUND_HALF_EVEN') BETWEEN 18 AND NUMERIC '24.99' AND upper(q.quest_type) = 'RI' AND q.version >= 6 AND substr(q.quest_ans, 1, 1) = '1' THEN real_risk_lvl"""


In [ ]:
prompt_template = '''help me convert this Teradata SQL to BigQuery SQL and end with two newline:
    {few_shot_prompt}
    Input:
    {qry}
    Output:'''

In [ ]:
for qry in parser.subqueries.values():
    prompt = prompt_template.format(few_shot_prompt=few_shot_prompt,
                                    qry=qry)

    # response_text = model.generate_text(prompt)
    # print(response_text)
    response_text = model.generate([prompt])
    print("\nresult:\n\n")
    for resp in response_text:
        print(resp.generated_text)

In [ ]:
prompt = f'''help me convert this Teradata SQL to BigQuery SQL and end with two newline:
{few_shot_prompt}
Input:
{sql}
Output:'''

# response_text = model.generate_text(prompt)
# print(response_text)
previous_output = ""
for i in range(5):
    prompt += previous_output
    response_text = model.generate([prompt])
    
    previous_output = ""
    for resp in response_text:
        previous_output += resp.generated_text
        print(resp.generated_text)